In [49]:
""" run_mots.py

Run example:
run_mots.py --USE_PARALLEL False --METRICS Hota --TRACKERS_TO_EVAL TrackRCNN

Command Line Arguments: Defaults, # Comments
    Eval arguments:
        'USE_PARALLEL': False,
        'NUM_PARALLEL_CORES': 8,
        'BREAK_ON_ERROR': True,
        'PRINT_RESULTS': True,
        'PRINT_ONLY_COMBINED': False,
        'PRINT_CONFIG': True,
        'TIME_PROGRESS': True,
        'OUTPUT_SUMMARY': True,
        'OUTPUT_DETAILED': True,
        'PLOT_CURVES': True,
    Dataset arguments:
        'GT_FOLDER': os.path.join(code_path, 'data/gt/mot_challenge/'),  # Location of GT data
        'TRACKERS_FOLDER': os.path.join(code_path, 'data/trackers/mot_challenge/'),  # Trackers location
        'OUTPUT_FOLDER': None,  # Where to save eval results (if None, same as TRACKERS_FOLDER)
        'TRACKERS_TO_EVAL': None,  # Filenames of trackers to eval (if None, all in folder)
        'CLASSES_TO_EVAL': ['pedestrian'],  # Valid: ['pedestrian']
        'SPLIT_TO_EVAL': 'train',  # Valid: 'train', 'test'
        'INPUT_AS_ZIP': False,  # Whether tracker input files are zipped
        'PRINT_CONFIG': True,  # Whether to print current config
        'TRACKER_SUB_FOLDER': 'data',  # Tracker files are in TRACKER_FOLDER/tracker_name/TRACKER_SUB_FOLDER
        'OUTPUT_SUB_FOLDER': '',  # Output files are saved in OUTPUT_FOLDER/tracker_name/OUTPUT_SUB_FOLDER
        'SEQMAP_FOLDER': None,  # Where seqmaps are found (if None, GT_FOLDER/seqmaps)
        'SEQMAP_FILE': None,  # Directly specify seqmap file (if none use seqmap_folder/MOTS-split_to_eval)
        'SEQ_INFO': None,  # If not None, directly specify sequences to eval and their number of timesteps
        'GT_LOC_FORMAT': '{gt_folder}/{seq}/gt/gt.txt',  # '{gt_folder}/{seq}/gt/gt.txt'
        'SKIP_SPLIT_FOL': False,    # If False, data is in GT_FOLDER/MOTS-SPLIT_TO_EVAL/ and in
                                    # TRACKERS_FOLDER/MOTS-SPLIT_TO_EVAL/tracker/
                                    # If True, then the middle 'MOTS-split' folder is skipped for both.
    Metric arguments:
        'METRICS': ['HOTA','CLEAR', 'Identity', 'VACE', 'JAndF']
"""

import sys
import os
import argparse
from multiprocessing import freeze_support
sys.path.append('../')
import trackeval  # noqa: E402


import numpy as np
from scipy.optimize import linear_sum_assignment


def hota_score(data):

    # Initialise results
    res = {}

    plottable = True
    array_labels = np.arange(0.05, 0.99, 0.05)
    integer_array_fields = ['HOTA_TP', 'HOTA_FN', 'HOTA_FP']
    float_array_fields = ['HOTA', 'DetA', 'AssA', 'DetRe', 'DetPr', 'AssRe', 'AssPr', 'LocA', 'RHOTA']
    float_fields = ['HOTA(0)', 'LocA(0)', 'HOTALocA(0)']
    fields = float_array_fields + integer_array_fields + float_fields
    summary_fields = float_array_fields + float_fields


    for field in float_array_fields + integer_array_fields:
        res[field] = np.zeros((len(array_labels)), dtype=np.float)
    for field in float_fields:
        res[field] = 0

    #tracker や gt シーケンスが空の場合、素早く結果を返す。
    if data['num_tracker_dets'] == 0:
        res['HOTA_FN'] = data['num_gt_dets'] * np.ones((len(array_labels)), dtype=np.float)
        res['LocA'] = np.ones((len(array_labels)), dtype=np.float)
        res['LocA(0)'] = 1.0
        return res
    if data['num_gt_dets'] == 0:
        res['HOTA_FP'] = data['num_tracker_dets'] * np.ones((len(array_labels)), dtype=np.float)
        res['LocA'] = np.ones((len(array_labels)), dtype=np.float)
        res['LocA(0)'] = 1.0
        return res

    #グローバルなアソシエーションをカウント
    potential_matches_count = np.zeros((data['num_gt_ids'], data['num_tracker_ids']))
    gt_id_count = np.zeros((data['num_gt_ids'], 1))
    tracker_id_count =  np.zeros((1, data['num_tracker_ids']))

    #最初に各タイムステップをループして、グローバルトラック情報を蓄積
    for t, (gt_ids_t, tracker_ids_t) in enumerate(zip(data['gt_ids'], data['tracker_ids'])):
        # Count the potential matches between ids in each timestep
        # These are normalised, weighted by the match similarity.
        similarity = data['similarity_scores'][t] #similarityは、gtとtrackerのボックスの重なりの割合
        sim_iou_denom = similarity.sum(0)[np.newaxis, :] + similarity.sum(1)[:, np.newaxis] - similarity

        sim_iou = np.zeros_like(similarity)
        sim_iou_mask = sim_iou_denom > 0 + np.finfo('float').eps
        sim_iou[sim_iou_mask] = similarity[sim_iou_mask] / sim_iou_denom[sim_iou_mask]
        potential_matches_count[gt_ids_t[:, np.newaxis], tracker_ids_t[np.newaxis, :]] += sim_iou

        ## gt_id と tracker_id ごとに dets の総数を計算。
        gt_id_count[gt_ids_t] += 1
        tracker_id_count[0, tracker_ids_t] += 1

    # ID間のjaccardアライメントスコア(ユニークマッチング前)を総合的に計算
    global_alignment_score = potential_matches_count / (gt_id_count + tracker_id_count - potential_matches_count)
    matches_counts = [np.zeros_like(potential_matches_count) for _ in array_labels]

    # Calculate scores for each timestep
    for t, (gt_ids_t, tracker_ids_t) in enumerate(zip(data['gt_ids'], data['tracker_ids'])):
        #タイムステップに gt_det/tracker_det が存在しないの処理
        if len(gt_ids_t) == 0:
            for a, alpha in enumerate(array_labels):
                res['HOTA_FP'][a] += len(tracker_ids_t)
            continue
        if len(tracker_ids_t) == 0:
            for a, alpha in enumerate(array_labels):
                res['HOTA_FN'][a] += len(gt_ids_t)
            continue

    #HOTAを最適化するためのペアのマッチングスコアを取得
    similarity = data['similarity_scores'][t]
    score_mat = global_alignment_score[gt_ids_t[:, np.newaxis], tracker_ids_t[np.newaxis, :]] * similarity

    # Hungarian algorithm
    match_rows, match_cols = linear_sum_assignment(-score_mat)

    for a, alpha in enumerate(array_labels):
        actually_matched_mask = similarity[match_rows, match_cols] >= alpha - np.finfo('float').eps
        alpha_match_rows = match_rows[actually_matched_mask]
        alpha_match_cols = match_cols[actually_matched_mask]
        num_matches = len(alpha_match_rows)
        res['HOTA_TP'][a] += num_matches
        res['HOTA_FN'][a] += len(gt_ids_t) - num_matches
        res['HOTA_FP'][a] += len(tracker_ids_t) - num_matches
        if num_matches > 0:
            res['LocA'][a] += sum(similarity[alpha_match_rows, alpha_match_cols])
            matches_counts[a][gt_ids_t[alpha_match_rows], tracker_ids_t[alpha_match_cols]] += 1

        #アルファ値に対するアソシエーションスコア(AssA, AssRe, AssPr)を算出
        #gt_id/tracker_idの組み合わせごとにスコアを計算し、検出数の平均をとる。
        for a, alpha in enumerate(array_labels):
            matches_count = matches_counts[a]
            ass_a = matches_count / np.maximum(1, gt_id_count + tracker_id_count - matches_count)
            res['AssA'][a] = np.sum(matches_count * ass_a) / np.maximum(1, res['HOTA_TP'][a])
            ass_re = matches_count / np.maximum(1, gt_id_count)
            res['AssRe'][a] = np.sum(matches_count * ass_re) / np.maximum(1, res['HOTA_TP'][a])
            ass_pr = matches_count / np.maximum(1, tracker_id_count)
            res['AssPr'][a] = np.sum(matches_count * ass_pr) / np.maximum(1, res['HOTA_TP'][a])
    
    # Calculate final scores
    res['LocA'] = np.maximum(1e-10, res['LocA']) / np.maximum(1e-10, res['HOTA_TP'])
    #Calculate sub-metric ('field') values which only depend on other sub-metric values.
    res['DetRe'] = res['HOTA_TP'] / np.maximum(1, res['HOTA_TP'] + res['HOTA_FN'])
    # res = self._compute_final_fields(res)
    res['DetPr'] = res['HOTA_TP'] / np.maximum(1, res['HOTA_TP'] + res['HOTA_FP'])
    res['DetA'] = res['HOTA_TP'] / np.maximum(1, res['HOTA_TP'] + res['HOTA_FN'] + res['HOTA_FP'])
    res['HOTA'] = np.sqrt(res['DetA'] * res['AssA'])
    res['RHOTA'] = np.sqrt(res['DetRe'] * res['AssA'])
    res['HOTA(0)'] = res['HOTA'][0]
    res['LocA(0)'] = res['LocA'][0]
    res['HOTALocA(0)'] = res['HOTA(0)']*res['LocA(0)']
    return res

def hota_eval_sequence(seq, dataset, tracker, class_list, metrics_list):
    """Function for evaluating a single sequence"""

    raw_data = dataset.get_raw_seq_data(tracker, seq)
    seq_res = {}
    for cls in class_list:
        seq_res[cls] = {}
        data = dataset.get_preprocessed_seq_data(raw_data, cls)
        seq_res[cls]['HOTA'] = hota_score(data)
        
    return seq_res

def _compute_final_fields(res):
    """Calculate sub-metric ('field') values which only depend on other sub-metric values.
    This function is used both for both per-sequence calculation, and in combining values across sequences.
    """
    res['DetRe'] = res['HOTA_TP'] / np.maximum(1, res['HOTA_TP'] + res['HOTA_FN'])
    res['DetPr'] = res['HOTA_TP'] / np.maximum(1, res['HOTA_TP'] + res['HOTA_FP'])
    res['DetA'] = res['HOTA_TP'] / np.maximum(1, res['HOTA_TP'] + res['HOTA_FN'] + res['HOTA_FP'])
    res['HOTA'] = np.sqrt(res['DetA'] * res['AssA'])
    res['RHOTA'] = np.sqrt(res['DetRe'] * res['AssA'])
    res['HOTA(0)'] = res['HOTA'][0]
    res['LocA(0)'] = res['LocA'][0]
    res['HOTALocA(0)'] = res['HOTA(0)']*res['LocA(0)']
    return res

def _combine_sum(all_res, field):
    """Combine sequence results via sum"""
    return sum([all_res[k][field] for k in all_res.keys()])

def _combine_weighted_av(all_res, field, comb_res, weight_field):
    """Combine sequence results via weighted average"""
    return sum([all_res[k][field] * all_res[k][weight_field] for k in all_res.keys()]) / np.maximum(1.0, comb_res[weight_field])

def combine_sequences(all_res):
    """Combines metrics across all sequences"""
    res = {}
    integer_array_fields = ['HOTA_TP', 'HOTA_FN', 'HOTA_FP']
    for field in integer_array_fields:
        res[field] = _combine_sum(all_res, field)
    for field in ['AssRe', 'AssPr', 'AssA']:
        res[field] = _combine_weighted_av(all_res, field, res, weight_field='HOTA_TP')
    loca_weighted_sum = sum([all_res[k]['LocA'] * all_res[k]['HOTA_TP'] for k in all_res.keys()])
    res['LocA'] = np.maximum(1e-10, loca_weighted_sum) / np.maximum(1e-10, res['HOTA_TP'])
    res = _compute_final_fields(res)
    return res

def combine_classes_class_averaged(self, all_res, ignore_empty_classes=False):
    """Combines metrics across all classes by averaging over the class values.
    If 'ignore_empty_classes' is True, then it only sums over classes with at least one gt or predicted detection.
    """
    res = {}
    plottable = True
    array_labels = np.arange(0.05, 0.99, 0.05)
    integer_array_fields = ['HOTA_TP', 'HOTA_FN', 'HOTA_FP']
    float_array_fields = ['HOTA', 'DetA', 'AssA', 'DetRe', 'DetPr', 'AssRe', 'AssPr', 'LocA', 'RHOTA']
    float_fields = ['HOTA(0)', 'LocA(0)', 'HOTALocA(0)']
    fields = float_array_fields + integer_array_fields + float_fields
    summary_fields = float_array_fields + float_fields

    for field in integer_array_fields:
        if ignore_empty_classes:
            res[field] = _combine_sum(
                {k: v for k, v in all_res.items()
                    if (v['HOTA_TP'] + v['HOTA_FN'] + v['HOTA_FP'] > 0 + np.finfo('float').eps).any()}, field)
        else:
            res[field] = _combine_sum({k: v for k, v in all_res.items()}, field)

    for field in float_fields + float_array_fields:
        if ignore_empty_classes:
            res[field] = np.mean([v[field] for v in all_res.values() if
                                    (v['HOTA_TP'] + v['HOTA_FN'] + v['HOTA_FP'] > 0 + np.finfo('float').eps).any()],
                                    axis=0)
        else:
            res[field] = np.mean([v[field] for v in all_res.values()], axis=0)
    return res

def combine_classes_det_averaged(self, all_res):
    """Combines metrics across all classes by averaging over the detection values"""
    res = {}
    plottable = True
    array_labels = np.arange(0.05, 0.99, 0.05)
    integer_array_fields = ['HOTA_TP', 'HOTA_FN', 'HOTA_FP']
    float_array_fields = ['HOTA', 'DetA', 'AssA', 'DetRe', 'DetPr', 'AssRe', 'AssPr', 'LocA', 'RHOTA']
    float_fields = ['HOTA(0)', 'LocA(0)', 'HOTALocA(0)']
    fields = float_array_fields + integer_array_fields + float_fields
    summary_fields = float_array_fields + float_fields

    for field in integer_array_fields:
        res[field] = _combine_sum(all_res, field)
    for field in ['AssRe', 'AssPr', 'AssA']:
        res[field] = _combine_weighted_av(all_res, field, res, weight_field='HOTA_TP')
    loca_weighted_sum = sum([all_res[k]['LocA'] * all_res[k]['HOTA_TP'] for k in all_res.keys()])
    res['LocA'] = np.maximum(1e-10, loca_weighted_sum) / np.maximum(1e-10, res['HOTA_TP'])
    res = _compute_final_fields(res)
    return res

In [50]:
# if __name__ == '__main__':
freeze_support()

# Command line interface:
default_eval_config = trackeval.Evaluator.get_default_eval_config()
default_eval_config['DISPLAY_LESS_PROGRESS'] = False
default_dataset_config = trackeval.datasets.MOTSChallenge.get_default_dataset_config()
default_metrics_config = {'METRICS': ['HOTA', 'CLEAR', 'Identity']}
config = {**default_eval_config, **default_dataset_config, **default_metrics_config}  # Merge default configs
parser = argparse.ArgumentParser()
for setting in config.keys():
    if type(config[setting]) == list or type(config[setting]) == type(None):
        parser.add_argument("--" + setting, nargs='+')
    else:
        parser.add_argument("--" + setting)
args = parser.parse_args(args=[]).__dict__
for setting in args.keys():
    if args[setting] is not None:
        if type(config[setting]) == type(True):
            if args[setting] == 'True':
                x = True
            elif args[setting] == 'False':
                x = False
            else:
                raise Exception('Command line parameter ' + setting + 'must be True or False')
        elif type(config[setting]) == type(1):
            x = int(args[setting])
        elif type(args[setting]) == type(None):
            x = None
        elif setting == 'SEQ_INFO':
            x = dict(zip(args[setting], [None]*len(args[setting])))
        else:
            x = args[setting]
        config[setting] = x
eval_config = {k: v for k, v in config.items() if k in default_eval_config.keys()}
dataset_config = {k: v for k, v in config.items() if k in default_dataset_config.keys()}
metrics_config = {k: v for k, v in config.items() if k in default_metrics_config.keys()}


# Run code
evaluator = trackeval.Evaluator(eval_config)
dataset_list = [trackeval.datasets.MOTSChallenge(dataset_config)]
metrics_list = []
for metric in [trackeval.metrics.HOTA, trackeval.metrics.CLEAR, trackeval.metrics.Identity, trackeval.metrics.VACE,
                trackeval.metrics.JAndF]:
    if metric.get_name() in metrics_config['METRICS']:
        metrics_list.append(metric())
if len(metrics_list) == 0:
    raise Exception('No metrics selected for evaluation')


Eval Config:
USE_PARALLEL         : False                         
NUM_PARALLEL_CORES   : 8                             
BREAK_ON_ERROR       : True                          
RETURN_ON_ERROR      : False                         
LOG_ON_ERROR         : /home/guest/dev_repo/SoccerTrack/sandbox/error_log.txt
PRINT_RESULTS        : True                          
PRINT_ONLY_COMBINED  : False                         
PRINT_CONFIG         : True                          
TIME_PROGRESS        : True                          
DISPLAY_LESS_PROGRESS : False                         
OUTPUT_SUMMARY       : True                          
OUTPUT_EMPTY_CLASSES : True                          
OUTPUT_DETAILED      : True                          
PLOT_CURVES          : True                          

MOTSChallenge Config:
PRINT_CONFIG         : True                          
GT_FOLDER            : /home/guest/dev_repo/SoccerTrack/sandbox/../data/trackeval/gt/mot_challenge/
TRACKERS_FOLDER      : /home

In [51]:
dataset_list

In [52]:
import time
import traceback
from multiprocessing.pool import Pool
from functools import partial
import os
# from . import utils
from trackeval.utils import TrackEvalException
# from . import _timing
from trackeval.metrics import Count




# def evaluate(self, dataset_list, metrics_list):
"""Evaluate a set of metrics on a set of datasets"""
# config = self.config

metric_name = 'HOTA'
# metrics_list = metrics_list + [Count()]  # Count metrics are always run
# metric_names = trackeval.utils.validate_metrics_list(metrics_list)
dataset_names = [dataset.get_name() for dataset in dataset_list]
output_res = {}
output_msg = {}

for dataset, dataset_name in zip(dataset_list, dataset_names):
    # Get dataset info about what to evaluate
    output_res[dataset_name] = {}
    output_msg[dataset_name] = {}
    tracker_list, seq_list, class_list = dataset.get_eval_info()
    print('\nEvaluating %i tracker(s) on %i sequence(s) for %i class(es) on %s dataset using the following '
            'metrics: %s\n' % (len(tracker_list), len(seq_list), len(class_list), dataset_name,
                                ', '.join(metric_name)))
    
    for tracker in tracker_list:
        # if not config['BREAK_ON_ERROR'] then go to next tracker without breaking
        try:
        # Evaluate each sequence in parallel or in series.
        # returns a nested dict (res), indexed like: res[seq][class][metric_name][sub_metric field]
        # e.g. res[seq_0001][pedestrian][hota][DetA]
            print('\nEvaluating %s\n' % tracker)
            time_start = time.time()
            if config['USE_PARALLEL']:
                with Pool(config['NUM_PARALLEL_CORES']) as pool:
                    _eval_sequence = partial(eval_sequence, dataset=dataset, tracker=tracker,
                                                class_list=class_list, metrics_list=metrics_list,
                                                metric_names=metric_names)
                    results = pool.map(_eval_sequence, seq_list)
                    res = dict(zip(seq_list, results))
            else:
                res = {}
                for curr_seq in sorted(seq_list):
                        res[curr_seq] = hota_eval_sequence(curr_seq, dataset, tracker, class_list, metrics_list)

            # Combine results over all sequences and then over all classes

            # collecting combined cls keys (cls averaged, det averaged, super classes)
            combined_cls_keys = []
            res['COMBINED_SEQ'] = {}
            # combine sequences for each class
            for c_cls in class_list:
                res['COMBINED_SEQ'][c_cls] = {}
                ######
                # 変更#
                ######
                # for metric, metric_name in zip(metrics_list, metric_names):
                curr_res = {seq_key: seq_value[c_cls]['HOTA'] for seq_key, seq_value in res.items() if
                                seq_key != 'COMBINED_SEQ'}
                res['COMBINED_SEQ'][c_cls]['HOTA'] = combine_sequences(curr_res)
            # combine classes
            if dataset.should_classes_combine:
                combined_cls_keys += ['cls_comb_cls_av', 'cls_comb_det_av', 'all']
                res['COMBINED_SEQ']['cls_comb_cls_av'] = {}
                res['COMBINED_SEQ']['cls_comb_det_av'] = {}

                # for metric, metric_name in zip(metrics_list, metric_names):

                cls_res = {cls_key: cls_value[metric_name] for cls_key, cls_value in
                                res['COMBINED_SEQ'].items() if cls_key not in combined_cls_keys}
                res['COMBINED_SEQ']['cls_comb_cls_av'][metric_name] = \
                        combine_classes_class_averaged(cls_res)
                res['COMBINED_SEQ']['cls_comb_det_av'][metric_name] = \
                        combine_classes_det_averaged(cls_res)

            # Combine results over all sequences and then over all classes
            if dataset.use_super_categories:
                combined_cls_keys += ['cls_comb_cls_av', 'cls_comb_det_av', 'all']
                res['COMBINED_SEQ']['cls_comb_cls_av'] = {}
                res['COMBINED_SEQ']['cls_comb_det_av'] = {}
                # for metric, metric_name in zip(metrics_list, metric_names):
                cls_res = {cls_key: cls_value[metric_name] for cls_key, cls_value in
                                res['COMBINED_SEQ'].items() if cls_key not in combined_cls_keys}
                res['COMBINED_SEQ']['cls_comb_cls_av'][metric_name] = \
                        combine_classes_class_averaged(cls_res)
                res['COMBINED_SEQ']['cls_comb_det_av'][metric_name] = \
                        combine_classes_det_averaged(cls_res)

            # combine classes to super classes
            if dataset.use_super_categories:
                for cat, sub_cats in dataset.super_categories.items():
                    combined_cls_keys.append(cat)
                    res['COMBINED_SEQ'][cat] = {}
                    # for metric, metric_name in zip(metrics_list, metric_names):
                    cat_res = {cls_key: cls_value[metric_name] for cls_key, cls_value in
                                    res['COMBINED_SEQ'].items() if cls_key in sub_cats}
                    res['COMBINED_SEQ'][cat][metric_name] = combine_classes_det_averaged(cat_res)
        
            # # Print and output results in various formats
            # if config['TIME_PROGRESS']:
            #     print('\nAll sequences for %s finished in %.2f seconds' % (tracker, time.time() - time_start))
            # output_fol = dataset.get_output_fol(tracker)
            # tracker_display_name = dataset.get_display_name(tracker)
            # for c_cls in res['COMBINED_SEQ'].keys():  # class_list + combined classes if calculated
            #     summaries = []
            #     details = []
            #     num_dets = res['COMBINED_SEQ'][c_cls]['Count']['Dets']
            #     if config['OUTPUT_EMPTY_CLASSES'] or num_dets > 0:
            #         for metric, metric_name in zip(metrics_list, metric_names):
            #             # for combined classes there is no per sequence evaluation
            #             if c_cls in combined_cls_keys:
            #                 table_res = {'COMBINED_SEQ': res['COMBINED_SEQ'][c_cls][metric_name]}
            #             else:
            #                 table_res = {seq_key: seq_value[c_cls][metric_name] for seq_key, seq_value
            #                                 in res.items()}

            #             if config['PRINT_RESULTS'] and config['PRINT_ONLY_COMBINED']:
            #                 dont_print = dataset.should_classes_combine and c_cls not in combined_cls_keys
            #                 if not dont_print:
            #                     metric.print_table({'COMBINED_SEQ': table_res['COMBINED_SEQ']},
            #                                         tracker_display_name, c_cls)
            #             elif config['PRINT_RESULTS']:
            #                 metric.print_table(table_res, tracker_display_name, c_cls)
            #             if config['OUTPUT_SUMMARY']:
            #                 summaries.append(metric.summary_results(table_res))
            #             if config['OUTPUT_DETAILED']:
            #                 details.append(metric.detailed_results(table_res))
            #             if config['PLOT_CURVES']:
            #                 metric.plot_single_tracker_results(table_res, tracker_display_name, c_cls,
            #                                                     output_fol)
            #         if config['OUTPUT_SUMMARY']:
            #             utils.write_summary_results(summaries, c_cls, output_fol)
            #         if config['OUTPUT_DETAILED']:
            #             utils.write_detailed_results(details, c_cls, output_fol)
            
            # Output for returning from function
            output_res[dataset_name][tracker] = res
            output_msg[dataset_name][tracker] = 'Success'
        
        # except KeyError:
        #     print('key error, skipping')
        
        except Exception as err:
            output_res[dataset_name][tracker] = None
            if type(err) == TrackEvalException:
                output_msg[dataset_name][tracker] = str(err)
            else:
                output_msg[dataset_name][tracker] = 'Unknown error occurred.'
            print('Tracker %s was unable to be evaluated.' % tracker)
            print(err)
            traceback.print_exc()
            if config['LOG_ON_ERROR'] is not None:
                with open(config['LOG_ON_ERROR'], 'a') as f:
                    print(dataset_name, file=f)
                    print(tracker, file=f)
                    print(traceback.format_exc(), file=f)
                    print('\n\n\n', file=f)


    #         if config['BREAK_ON_ERROR']:
    #             raise err
    #         elif config['RETURN_ON_ERROR']:
    #             return output_res, output_msg
    
    # return output_res, output_msg


Evaluating 1 tracker(s) on 4 sequence(s) for 1 class(es) on MOTSChallenge dataset using the following metrics: H, O, T, A


Evaluating track_rcnn

    MOTSChallenge.get_raw_seq_data(track_rcnn, MOTS20-02)                  0.9761 sec
    MOTSChallenge.get_preprocessed_seq_data(pedestrian)                    0.2448 sec


/tmp/ipykernel_5154/2033258168.py:67: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  res[field] = np.zeros((len(array_labels)), dtype=np.float)


    MOTSChallenge.get_raw_seq_data(track_rcnn, MOTS20-05)                  0.5718 sec
    MOTSChallenge.get_preprocessed_seq_data(pedestrian)                    0.2764 sec
    MOTSChallenge.get_raw_seq_data(track_rcnn, MOTS20-09)                  0.7616 sec
    MOTSChallenge.get_preprocessed_seq_data(pedestrian)                    0.1839 sec
    MOTSChallenge.get_raw_seq_data(track_rcnn, MOTS20-11)                  1.2948 sec
    MOTSChallenge.get_preprocessed_seq_data(pedestrian)                    0.3014 sec


In [53]:
output_res['MOTSChallenge']['track_rcnn']['COMBINED_SEQ']

{'pedestrian': {'HOTA': {'HOTA_TP': array([32., 32., 32., 32., 32., 32., 32., 31., 30., 30., 28., 27., 25.,
          22., 16., 12.,  5.,  1.,  0.]),
   'HOTA_FN': array([ 8.,  8.,  8.,  8.,  8.,  8.,  8.,  9., 10., 10., 12., 13., 15.,
          18., 24., 28., 35., 39., 40.]),
   'HOTA_FP': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  2.,  2.,  4.,  5.,  7.,
          10., 16., 20., 27., 31., 32.]),
   'AssRe': array([0.01607564, 0.01607564, 0.01607564, 0.01607564, 0.01607564,
          0.01607564, 0.01607564, 0.01636543, 0.01653215, 0.01653215,
          0.01676756, 0.01658343, 0.01733161, 0.0192022 , 0.01746163,
          0.01886974, 0.03080807, 0.03030303, 0.        ]),
   'AssPr': array([0.0240342 , 0.0240342 , 0.0240342 , 0.0240342 , 0.0240342 ,
          0.0240342 , 0.0240342 , 0.0239606 , 0.02466267, 0.02466267,
          0.02479594, 0.01954147, 0.02069034, 0.02144815, 0.01147926,
          0.01281836, 0.01427178, 0.00383142, 0.        ]),
   'AssA': array([0.00646085, 0.0064